In [ ]:
%matplotlib inline

import random
import numpy as np
import matplotlib.pyplot as plt


#------------------------------------------------------------------------------------------------------------------------
#Generating n points at random coordinates and return them in a list

def pointGeneration(nb,scaleMultiplier):
    n = nb
    scale = scaleMultiplier*n
    depot = (scale/2,scale/2)
    pointList = [depot]
    
    for i in range(n):
        pointList += (i,)
        pointList[i+1] = (random.randrange(0,scale),random.randrange(0,scale))
        
    return pointList

#------------------------------------------------------------------------------------------------------------------------
#Display all the points on the graph

def plotPoints(pointList):
    plt.plot(pointList[0][0],pointList[0][1],"ro", markersize=15)
    plt.annotate("dépot", (pointList[0][0]+3, pointList[0][1]+3), fontsize=10)
    
    for i in range(len(pointList)):
        if (i!= 0):
            plt.plot(pointList[i][0], pointList[i][1],"bo")
            plt.annotate(i, (pointList[i][0]+3, pointList[i][1]+3), fontsize=12)
            
#------------------------------------------------------------------------------------------------------------------------
#Choose n points as delivery points, display them on the graph and return them in a list

def randomPackages(n, nbPointsMax):
    package = random.sample(range(1, nbPointsMax), n)
    
    for i in range(len(package)):
        plt.plot(pointList[package[i]][0], pointList[package[i]][1],"go", markersize=10)
        
    return package

#------------------------------------------------------------------------------------------------------------------------
#Creating the adjacency matrix so the graph is connected and return the matrix

def makematrix(liste, linksLimit):
    matrix = np.zeros((len(liste),len(liste)))
    
    for i in range(len(matrix)):
        matrix[i][(i+1)%(len(matrix))] = 1
        matrix[(i+1)%(len(matrix))][i] = 1
    
    for i in range(len(matrix)-1):
        
            nbchoisi = 0
            while(sum(matrix[i])<linksLimit and matrix[:, i].sum() < linksLimit):
                nbchoisi = np.random.randint(i,len(matrix))
                
                if (matrix[:, nbchoisi].sum() < linksLimit and sum(matrix[nbchoisi])<linksLimit):
                    matrix[i][nbchoisi] = 1
                    matrix[nbchoisi][i] = 1
                   
    for i in range(len(matrix)):
        matrix[i][i] = 0
        
    return matrix

#------------------------------------------------------------------------------------------------------------------------
#Display links between every neighbor points    

def TraceLinks(matrix):
    
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if(i>j):
                if matrix[i][j] > 0:
                    x = (pointList[i][0], pointList[j][0])
                    y = (pointList[i][1], pointList[j][1])
                    plt.plot(x, y,"b", linestyle="solid")
   
#------------------------------------------------------------------------------------------------------------------------
#Give weight to every links in the matrix depending on the coordinates difference of the points

def ponderation(pointList, matrix):
    
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if matrix[i][j] == 1:
                poids = np.sqrt((pointList[i][0]-pointList[j][0])**2+(pointList[i][1]-pointList[j][1])**2)
                matrix[i][j] = round(poids)
                
#------------------------------------------------------------------------------------------------------------------------

#------------------------------------------------------------------------------------------------------------------------
#Generate a random existing solution starting from the depot, including all delivery points and coming back

def generateSolution(matrix,package):

    packageLeft = package.copy()
    solution = [0]

    while(packageLeft):
        randomNext = np.random.randint(0,len(matrix))
        while( matrix[solution[-1]][randomNext] == 0 ):
            randomNext = np.random.randint(0,len(matrix))
        solution.append(randomNext)
        if(randomNext in packageLeft):
            packageLeft.remove(randomNext)
    
    while(solution[-1] != 0):
        randomNext = np.random.randint(0,len(matrix))
        while( matrix[solution[-1]][randomNext] == 0 ):
            randomNext = np.random.randint(0,len(matrix))
        solution.append(randomNext)
    
    return solution

#------------------------------------------------------------------------------------------------------------------------
#Return the length of the solution

def solutionLength(solution):
    
    totalLength = 0
    
    for i in range(0,len(solution)-1):
        totalLength += matrix[solution[i]][solution[i+1]]
        
    return totalLength

#------------------------------------------------------------------------------------------------------------------------
#Generate n random existing solutions and return them in a list

def generatePopulation(matrix,package,populationSize):

    population = []
    for i in range(populationSize):

        population.append(generateSolution(matrix,package))
        
    return population

#------------------------------------------------------------------------------------------------------------------------
#Return only the shortest half of solutions in a list

def selection(population):
    
    population2 = []
    
    for i in range(0,len(population)-1,2):
        if(solutionLength(population[i]) < solutionLength(population[i+1])):
            population2.append(population[i])
        else:
            population2.append(population[i+1])
            
    return population2
    
#------------------------------------------------------------------------------------------------------------------------
#Mutate the given population and return a new population composed of mutated + given population

def mutation(population2,matrix,package):
    
    mutatedPopu = []
    
    for i in range(len(population2)):
        mutatedSolu = population2[i].copy()
        mutatedPopu.append(population2[i].copy())
        mutationRate = np.random.randint(1,len(mutatedSolu)-1)
        del mutatedSolu[-mutationRate:]
        
        packageLeft = package.copy()

        for i in range(0,len(mutatedSolu)-1):
            if(mutatedSolu[i] in packageLeft):
                packageLeft.remove(mutatedSolu[i])

        while(packageLeft):
            randomNext = np.random.randint(0,len(matrix))
            while( matrix[mutatedSolu[-1]][randomNext] == 0 ):
                randomNext = np.random.randint(0,len(matrix))
            mutatedSolu.append(randomNext)
            if(randomNext in packageLeft):
                packageLeft.remove(randomNext)
            
        while(mutatedSolu[-1] != 0):
            randomNext = np.random.randint(0,len(matrix))
            while( matrix[mutatedSolu[-1]][randomNext] == 0 ):
                randomNext = np.random.randint(0,len(matrix))
            mutatedSolu.append(randomNext)

        mutatedPopu.append(mutatedSolu)
    
    return mutatedPopu

#------------------------------------------------------------------------------------------------------------------------
#Apply selection and mutation n times and return the shortest solution

def iterate(nbIteration, matrix, package, populationSize):

    initPopulation = generatePopulation(matrix, package, populationSize)
    select = selection(initPopulation)
    
    for i in range(nbIteration):
        
        length = []
        muta = mutation(select,matrix, package)
        select = selection(muta)
        
        for j in range(len(muta)):
            length.append(solutionLength(muta[j]))
            
        print(i, ":",muta[length.index(min(length))])
        #print(i)
        #print("index :",length.index(min(length)))
        print("length :",min(length),"\n")
    
    return muta[length.index(min(length))]

#------------------------------------------------------------------------------------------------------------------------
#Display the solution on the graph

def traceSolution(s,package, pointList):
    plt.plot(pointList[0][0],pointList[0][1],"ro", markersize=15)
    plt.annotate("dépot", (pointList[0][0]+3, pointList[0][1]+3), fontsize=10)

    for i in range(len(package)):

        plt.plot(pointList[package[i]][0], pointList[package[i]][1],"go", markersize=10)

    for i in range(0,len(s)-1):
        x = (pointList[s[i]][0], pointList[s[i+1]][0])
        y = (pointList[s[i]][1], pointList[s[i+1]][1])
        plt.plot(x, y,"b", linestyle="solid")
        plt.annotate(s[i+1], (pointList[s[i+1]][0]+3, pointList[s[i+1]][1]+3), fontsize=12)

        
#------------------------------------------------------------------------------------------------------------------------
#Functions to remove uselsess back and forth in the solution of iterations

def verifPresenceElemtable1_In_table2(elemsAVerifier, TableauEntier):
    for e in elemsAVerifier:
        if(e in TableauEntier):
            return True
        
    return False  
    
    
def verif_Not_Useless_Path(solution, package):
    for i in solution:
        if solution.count(i) > 1:
            occurs = [j for j, x in enumerate(solution) if x == i]
            for o in range(len(occurs)):
                if (o != len(occurs)-1):
                    if (not(verifPresenceElemtable1_In_table2(package,solution[occurs[o]+1:occurs[o+1]+1]))):
                        del solution[occurs[o]+1:occurs[o+1]+1]
                        return False
    return True

#--------------------------------------------------------------------------------------------------------------------------------------------
            

def remove_useless_subpaths(solution, package):
    verif = False
    while(verif == False):#REMPLACER PAR WHILE
        verif = verif_Not_Useless_Path(solution, package)
        
    return solution


In [ ]:
nbPoints = 150
nbVoisins = 3
nbPackage = 7
scaleMultiplier = 10

nbIteration = 10
populationSize = 40



pointList = pointGeneration(nbPoints,scaleMultiplier)
plotPoints(pointList)
#print("Liste des points :", pointList)

package = randomPackages(nbPackage, nbPoints)
#print("Liste des paquets :", package)

matrix = makematrix(pointList, nbVoisins)
print("Matrice d'adjacence \n", matrix)

ponderation (pointList,matrix)
#print("Matrice d'adjacence pondérée \n", matrix)

TraceLinks(matrix)


In [ ]:
%%time
s = iterate(nbIteration,matrix,package, populationSize)
print(s)
print(solutionLength(s))

In [ ]:
traceSolution(s,package,pointList)
print(package)

In [ ]:
print(s)
print(solutionLength(s))

print(remove_useless_subpaths(s, package))
print(solutionLength(remove_useless_subpaths(s, package)))

In [ ]:
traceSolution(remove_useless_subpaths(s, package),package, pointList)
print(package)